In [49]:
# IMport the dependencies.
import pandas as pd
import gmaps 
import requests

# Import the API key.
from config import g_key

In [50]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")

In [55]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [56]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [59]:
# Assign the locations to an array of latitude and logitude pairs.
locations = city_data_df[["Lat", "Lng"]]

# Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap_layer variable to the heatmap_layer and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [60]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [61]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [62]:
# Heatmap of Wind Speed.
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [67]:
# Ask the customer to add minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [68]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Tabou,CI,2022-04-26 19:15:38,4.4230,-7.3528,81.43,81,100,6.11
12,12,Arraial Do Cabo,BR,2022-04-26 19:17:40,-22.9661,-42.0278,82.36,83,0,16.11
20,20,San Vicente,AR,2022-04-26 19:17:43,-26.6167,-54.1333,86.13,48,3,6.29
23,23,San Rafael,AR,2022-04-26 19:17:44,-34.6177,-68.3301,82.51,7,0,17.02
33,33,Avarua,CK,2022-04-26 19:17:48,-21.2078,-159.7750,80.65,89,20,3.44


In [69]:
preferred_cities_df.count()

City_ID       121
City          121
Country       121
Date          121
Lat           121
Lng           121
Max Temp      121
Humidity      121
Cloudiness    121
Wind Speed    121
dtype: int64

In [71]:
# Create a DataFrame called hotel_df to store hotel names along with city, country, max temp and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

hotel_df.head()

C:\Users\Bryan\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Tabou,CI,81.43,4.4230,-7.3528,
12,Arraial Do Cabo,BR,82.36,-22.9661,-42.0278,
20,San Vicente,AR,86.13,-26.6167,-54.1333,
23,San Rafael,AR,82.51,-34.6177,-68.3301,
33,Avarua,CK,80.65,-21.2078,-159.7750,


In [80]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [81]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Tabou,CI,81.43,4.4230,-7.3528,hotêl doufoulougou
12,Arraial Do Cabo,BR,82.36,-22.9661,-42.0278,Pousada Porto Praia
20,San Vicente,AR,86.13,-26.6167,-54.1333,Complejo Turístico Farelluk
23,San Rafael,AR,82.51,-34.6177,-68.3301,Hotel San Rafael
33,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...
565,Altar,MX,92.59,30.7000,-111.8167,Hotel Maria Elena
566,Bathsheba,BB,81.21,13.2167,-59.5167,Atlantis Hotel
569,Puerto Carreno,CO,91.38,6.1890,-67.4859,Casa Hotel Victoria
575,Karwar,IN,81.52,14.8000,74.1333,Netalco Connects


In [83]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [86]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [87]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))